# 10.1 GroupBy framework

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({"key1": ["a", "a", None, "b", "b", "a", None],
                   "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1": np.random.standard_normal(7),
                   "data2": np.random.standard_normal(7)})

In [3]:
df

,key1,key2,data1,data2
0,a,1,-0.548535,0.743117
1,a,2,0.226126,-0.212000
2,None,1,-1.309400,0.099571
3,b,2,-1.046801,1.354038
4,b,1,-1.833910,-0.927567
5,a,<NA>,0.844650,1.148555
6,None,1,-1.371766,2.641112


In [4]:
grouped = df["data1"].groupby(df["key1"])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a    0.174080
b   -1.440356
Name: data1, dtype: float64

In [7]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()

In [8]:
means

key1  key2
a     1      -0.548535
      2       0.226126
b     1      -1.833910
      2      -1.046801
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,1,2
key1,,
a,-0.548535,0.226126
b,-1.833910,-1.046801


In [10]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])

In [11]:
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [12]:
df["data1"].groupby([states, years]).mean()

CA  2005    0.535388
    2006   -1.309400
OH  2005   -0.797668
    2006   -1.602838
Name: data1, dtype: float64

In [13]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.174080,0.559890
b,1.5,-1.440356,0.213235


In [14]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.548535  0.743117
     2     0.226126 -0.212000
b    1    -1.833910 -0.927567
     2    -1.046801  1.354038

In [15]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [16]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [17]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [18]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


In [19]:
df.groupby("key2").count()

,key1,data1,data2
key2,,,
1,2,4,4
2,2,2,2


# 10.1.1 iterate on grouping

In [20]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.548535  0.743117
1    a     2  0.226126 -0.212000
5    a  <NA>  0.844650  1.148555
b
  key1  key2     data1     data2
3    b     2 -1.046801  1.354038
4    b     1 -1.833910 -0.927567


In [21]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1     data2
0    a     1 -0.548535  0.743117
('a', np.int64(2))
  key1  key2     data1  data2
1    a     2  0.226126 -0.212
('b', np.int64(1))
  key1  key2    data1     data2
4    b     1 -1.83391 -0.927567
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2 -1.046801  1.354038


In [22]:
pieces = {name: group for name, group in df.groupby("key1")}

In [23]:
pieces["b"]

,key1,key2,data1,data2
3,b,2,-1.046801,1.354038
4,b,1,-1.833910,-0.927567


In [24]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

/tmp/ipykernel_3725/1884927282.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


In [25]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.548535  0.743117
1  0.226126 -0.212000
2 -1.309400  0.099571
3 -1.046801  1.354038
4 -1.833910 -0.927567
5  0.844650  1.148555
6 -1.371766  2.641112
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


# 10.1.2 select one or more columns

In [26]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.743117
     2    -0.212000
b    1    -0.927567
     2     1.354038

In [27]:
s_grouped = df.groupby(["key1", "key2"])["data2"]

In [28]:
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     1       0.743117
      2      -0.212000
b     1      -0.927567
      2       1.354038
Name: data2, dtype: float64

# 10.1.3 grouping using dictionaries and series 

In [30]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

In [31]:
people.iloc[2:3, [1, 2]] = np.nan #join some none

In [32]:
people

,a,b,c,d,e
Joe,0.151892,0.407128,0.585773,0.386329,-0.459581
Steve,0.175744,-1.133199,-2.240784,0.067369,-0.031998
Wanda,1.551732,NaN,NaN,0.645852,0.165430
Jill,0.356923,1.638970,-0.057110,0.828071,0.481303
Trey,0.062873,0.942355,0.062359,-0.027706,0.169985


In [33]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f": "orange"}

In [34]:
by_column = people.groupby(mapping, axis="columns")

/tmp/ipykernel_3725/2536829432.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


In [35]:
by_column.sum()

,blue,red
Joe,0.972102,0.099438
Steve,-2.173414,-0.989453
Wanda,0.645852,1.717162
Jill,0.770961,2.477196
Trey,0.034653,1.175213


In [36]:
map_series = pd.Series(mapping)

In [37]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [38]:
people.groupby(map_series, axis="columns").count()

/tmp/ipykernel_3725/2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


# 10.1.4 grouping using functions

In [39]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.151892,0.407128,0.585773,0.386329,-0.459581
4,0.419796,2.581325,0.005249,0.800365,0.651287
5,1.727476,-1.133199,-2.240784,0.713221,0.133432


In [40]:
key_list = ["one", "one", "one", "two", "two"]

In [41]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.151892,0.407128,0.585773,0.386329,-0.459581
4,two,0.062873,0.942355,-0.057110,-0.027706,0.169985
5,one,0.175744,-1.133199,-2.240784,0.067369,-0.031998


# 10.1.5 grouping according to index level

In [42]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])

In [43]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)

In [44]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.223472  0.231542  0.079538  2.335178  1.217613
1     -0.882411 -0.735598  0.697072 -0.406279  1.111672
2     -0.292259  0.616389  1.513398  0.357731  0.382980
3     -0.581906  0.139620 -2.035136 -1.468816  2.046003

In [45]:
hier_df.groupby(level="cty", axis="columns").count()

/tmp/ipykernel_3725/2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 data aggregation

In [46]:
df

,key1,key2,data1,data2
0,a,1,-0.548535,0.743117
1,a,2,0.226126,-0.212000
2,None,1,-1.309400,0.099571
3,b,2,-1.046801,1.354038
4,b,1,-1.833910,-0.927567
5,a,<NA>,0.844650,1.148555
6,None,1,-1.371766,2.641112


In [47]:
grouped = df.groupby("key1")

In [48]:
grouped["data1"].nsmallest(2)

key1   
a     0   -0.548535
      1    0.226126
b     4   -1.833910
      3   -1.046801
Name: data1, dtype: float64

In [49]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [50]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.393185,1.360555
b,1,0.787109,2.281605


In [51]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.174080  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -1.440356  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.535388  0.844650   3.0  0.559890  0.698539 -0.212000  0.265558   
b    -1.243579 -1.046801   2.0  0.213235  1.613338 -0.927567 -0.357166   

                                    
           50%       75%       max  
key1                                
a     0.743117  0.945836  1.148555  
b     0.213235  0.783636  1.354038  

[2 rows x 24 columns]

# 10.2.1 column by column operations and multi function applications

In [52]:
tips = pd.read_csv("examples/tips.csv")

In [53]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [54]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

In [55]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [56]:
grouped = tips.groupby(["day", "smoker"])

In [57]:
grouped_pct = grouped["tip_pct"]

In [58]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [59]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [60]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

/tmp/ipykernel_3725/1734782830.py:1: FutureWarning: The provided callable <function std at 0x70d7fdf8e170> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [61]:
functions = ["count", "mean", "max"]

In [62]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)

In [63]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [64]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [65]:
ftuples = [("Average", "mean"), ("Variance", np.var)]

In [66]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

/tmp/ipykernel_3725/365474927.py:1: FutureWarning: The provided callable <function var at 0x70d7fdf8e290> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [67]:
grouped.agg({"tip": np.max, "size": "sum"})

/tmp/ipykernel_3725/3453032214.py:1: FutureWarning: The provided callable <function max at 0x70d7fdf8d750> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

# 10.2.2 return aggregated data without row index

In [69]:
# tips.groupby(["day", "smoker"], as_index=False).mean()

In [70]:
tips.any()

total_bill    True
tip           True
smoker        True
day           True
time          True
size          True
tip_pct       True
dtype: bool

# 10.3 Apply: universal "split application union" specification

In [71]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [72]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [73]:
tips.groupby("smoker").apply(top)

/tmp/ipykernel_3725/2530541573.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [74]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

/tmp/ipykernel_3725/643835440.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [75]:
result = tips.groupby("smoker")["tip_pct"].describe()

In [76]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [77]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [78]:
def f(group):
    return group.describe()

In [79]:
grouped.apply(f)

/tmp/ipykernel_3725/833689054.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(f)


total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

# 10.3.1 disable grouping key

In [80]:
tips.groupby("smoker", group_keys=False).apply(top)

/tmp/ipykernel_3725/3851957478.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


# 10.3.2 quantile and bucket analysis

In [81]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})

In [82]:
frame.head()

,data1,data2
0,-0.375957,1.569349
1,-0.646045,0.341269
2,-0.014752,-0.972332
3,-0.285848,0.851569
4,1.315934,0.281211


In [83]:
quartiles = pd.cut(frame["data1"], 4)

In [84]:
quartiles.head(10)

0    (-1.659, -0.0544]
1    (-1.659, -0.0544]
2     (-0.0544, 1.551]
3    (-1.659, -0.0544]
4     (-0.0544, 1.551]
5    (-1.659, -0.0544]
6    (-1.659, -0.0544]
7     (-0.0544, 1.551]
8     (-3.271, -1.659]
9    (-1.659, -0.0544]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.271, -1.659] < (-1.659, -0.0544] < (-0.0544, 1.551] < (1.551, 3.155]]

In [85]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
         "count": group.count(), "mead": group.mean()}
    )

In [86]:
grouped = frame.groupby(quartiles)

/tmp/ipykernel_3725/4041437760.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


In [87]:
grouped.apply(get_stats)

min       max  count      mead
data1                                                       
(-3.271, -1.659]  data1 -3.264362 -1.663180     62 -2.040386
                  data2 -1.563134  2.748197     62  0.084104
(-1.659, -0.0544] data1 -1.650289 -0.058845    415 -0.687855
                  data2 -2.427632  3.157584    415 -0.034751
(-0.0544, 1.551]  data1 -0.052697  1.548189    462  0.612629
                  data2 -2.406883  2.360664    462 -0.021395
(1.551, 3.155]    data1  1.550598  3.155479     61  1.961941
                  data2 -2.355567  2.681321     61  0.116068

In [88]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                        min       max count      mean       min       max   
data1                                                                       
(-3.271, -1.659]  -3.264362 -1.663180    62 -2.040386 -1.563134  2.748197   
(-1.659, -0.0544] -1.650289 -0.058845   415 -0.687855 -2.427632  3.157584   
(-0.0544, 1.551]  -0.052697  1.548189   462  0.612629 -2.406883  2.360664   
(1.551, 3.155]     1.550598  3.155479    61  1.961941 -2.355567  2.681321   

                                   
                  count      mean  
data1                              
(-3.271, -1.659]     62  0.084104  
(-1.659, -0.0544]   415 -0.034751  
(-0.0544, 1.551]    462 -0.021395  
(1.551, 3.155]       61  0.116068

In [89]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)

In [90]:
quartiles_samp.head()

0    1
1    1
2    1
3    1
4    3
Name: data1, dtype: int64

In [91]:
grouped = frame.groupby(quartiles_samp)

In [92]:
grouped.apply(get_stats)

min       max  count      mead
data1                                           
0     data1 -3.264362 -0.696777    250 -1.314400
      data2 -2.427632  3.157584    250 -0.009131
1     data1 -0.691849  0.005256    250 -0.335319
      data2 -2.315347  2.858765    250 -0.041839
2     data1  0.006874  0.693146    250  0.339682
      data2 -2.240016  2.360664    250  0.015840
3     data1  0.694381  3.155479    250  1.273034
      data2 -2.406883  2.681321    250 -0.012915

# 10.3.3 example: fill in missing values with values from the specified group

In [93]:
s = pd.Series(np.random.standard_normal(6))

In [94]:
s[::2] = np.nan

In [95]:
s

0         NaN
1   -0.265203
2         NaN
3   -0.366605
4         NaN
5   -0.114941
dtype: float64

In [96]:
s.fillna(s.mean())

0   -0.248916
1   -0.265203
2   -0.248916
3   -0.366605
4   -0.248916
5   -0.114941
dtype: float64

In [97]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

In [98]:
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

In [99]:
data = pd.Series(np.random.standard_normal(8), index=states)

In [100]:
data

Ohio         -1.545141
New York      0.494878
Vermont       0.009216
Florida      -0.634055
Oregon        0.504808
Nevada        0.653455
California    1.258845
Idaho        -0.686599
dtype: float64

In [101]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan

In [102]:
data

Ohio         -1.545141
New York      0.494878
Vermont            NaN
Florida      -0.634055
Oregon        0.504808
Nevada             NaN
California    1.258845
Idaho              NaN
dtype: float64

In [103]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [104]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [105]:
data.groupby(group_key).mean()

East   -0.561439
West    0.881826
dtype: float64

In [106]:
def fill_mean(group):
    return group.fillna(group.mean())

In [107]:
data.groupby(group_key).apply(fill_mean)

East  Ohio         -1.545141
      New York      0.494878
      Vermont      -0.561439
      Florida      -0.634055
West  Oregon        0.504808
      Nevada        0.881826
      California    1.258845
      Idaho         0.881826
dtype: float64

In [108]:
fill_values = {"East": 0.5, "West": -1}

In [109]:
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [110]:
data.groupby(group_key).apply(fill_func)

East  Ohio         -1.545141
      New York      0.494878
      Vermont       0.500000
      Florida      -0.634055
West  Oregon        0.504808
      Nevada       -1.000000
      California    1.258845
      Idaho        -1.000000
dtype: float64

# 10.3.4 example: random sampling and sorting

In [111]:
suits = ["H", "S", "C", "D"] # heart spade club diamond
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [112]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [113]:
def draw(deck, n=5):
    return deck.sample(n)

In [114]:
draw(deck)

4H      4
10S    10
9S      9
KH     10
AH      1
dtype: int64

In [115]:
def get_suit(card):
    # The last letter is the color
    return card[-1]

In [116]:
deck.groupby(get_suit).apply(draw, n=2)

C  3C      3
   10C    10
D  4D      4
   AD      1
H  KH     10
   JH     10
S  3S      3
   KS     10
dtype: int64

In [117]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

3C      3
5C      5
QD     10
5D      5
AH      1
6H      6
10S    10
JS     10
dtype: int64

# 10.3.5 example: group weighted average and correlation coefficient

In [118]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})

In [119]:
df

,category,data,weights
0,a,1.640502,0.863387
1,a,0.245334,0.743918
2,a,0.787774,0.568425
3,a,0.493222,0.192488
4,b,0.626122,0.398509
5,b,-0.107008,0.671029
6,b,-2.087577,0.399251
7,b,-0.285980,0.989935


In [120]:
grouped = df.groupby("category")

In [121]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

In [122]:
grouped.apply(get_wavg)

/tmp/ipykernel_3725/4286695940.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


category
a    0.904320
b   -0.381849
dtype: float64

In [123]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)

In [124]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [125]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [126]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [127]:
rets = close_px.pct_change().dropna()

In [128]:
def get_year(x):
    return x.year

In [129]:
by_year = rets.groupby(get_year)

In [130]:
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [131]:
by_year = rets.groupby(get_year)

In [132]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [133]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

In [134]:
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 10.3.6 example: grouped linear regression

In [135]:
import statsmodels.api as sm

In [136]:
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [137]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 gourp change and “expansion" GroupBy operation

In [138]:
df = pd.DataFrame({"key": ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})

In [139]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [140]:
g = df.groupby('key')['value']

In [141]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [142]:
def get_mean(group):
    return group.mean()

In [143]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [144]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [145]:
def times_two(group):
    return group * 2

In [146]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [147]:
def get_ranks(group):
    return group.rank(ascending=False)

In [148]:
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [149]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [150]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [151]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [152]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [153]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [154]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# 10.5 pivot table and cross table

In [155]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [156]:
tips.pivot_table(index=["day", "smoker"], values=["total_bill", "tip", "size", "tip_pct"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [158]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [159]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [160]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [162]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [163]:
from io import StringIO

In [164]:
data = """Sample Nationality Handedness
1 USA Right-handed
2 Japan Left-handed
3 USA Right-handed
4 Japan Right-handed
5 Japan Left-handed
6 Japan Right-handed
7 USA Right-handed
8 USA Left-handed
9 Japan Right-handed
10 USA Right-handed"""

In [165]:
data = pd.read_table(StringIO(data), sep="\s+")

In [166]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [167]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [168]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244